In [2]:
import numpy as np
import scipy.stats as sc
import matplotlib.pyplot as plt
from time import time

In [3]:
# ground truth data
ground_truths = np.load('mt_trajectories.npy')
ground_truths

array([[[ -8.07218076,  -4.49125581,  -0.91033086,   2.67059409,
           6.25151904,   9.83244399,  13.41336894,  16.99429389,
          20.57521884,  24.15614379],
        [ -0.8844742 ,  -2.77044353,  -4.65641285,  -6.54238217,
          -8.4283515 , -10.31432082, -12.20029014, -14.08625947,
         -15.97222879, -17.85819811],
        [ -3.88086752,  -2.92152956,  -1.96219159,  -1.00285363,
          -0.04351567,   0.91582229,   1.87516025,   2.83449821,
           3.79383618,   4.75317414]],

       [[ -6.46454434,  -3.36883308,  -0.27312181,   2.82258945,
           5.91830072,   9.01401198,  12.10972325,  15.20543452,
          18.30114578,  21.39685705],
        [  5.1703906 ,   6.05898913,   6.94758765,   7.83618618,
           8.72478471,   9.61338324,  10.50198177,  11.3905803 ,
          12.27917883,  13.16777736],
        [ -1.3581392 ,  -0.35850924,   0.64112072,   1.64075068,
           2.64038064,   3.6400106 ,   4.63964056,   5.63927052,
           6.63890048,   7.6

## Background

In [21]:
# # utility functions
def create_state_matrix(dt:float = 1.0, dim:int = 2):
    mat = []
    for i in range(dim):
        row = []
        for j in range(dim):
            if i == j:
                row.append(np.array([[1, dt], [0, 1]]))
            else:
                row.append(np.zeros((2, 2)))
        mat.append(row)
    return np.block(mat)

def create_measurement_matrix(dim:int = 2):
    mat = []
    for i in range(dim):
        row = []
        for j in range(dim):
            if i == j:
                row.append(np.array([1, 0]))
            else:
                row.append(np.zeros(2))
        mat.append(row)
    return np.block(mat)

The motion model describes how objects of a system move. Define the motion model as $$x_{k+1} = F \times x_{k} + w_{k}$$ where
\begin{gather*}
    \text{$x_{k+1}$: next state} \\
    \text{$F$: state transition matrix} \\
    \text{$x_{k}$: current state} \\
    \text{$w_{k}$: process noise.} \\
\end{gather*}

Assume motion is constant velocity in 2 dimensions, where state is represented by $[x, v_{x}, y, v_{y}]$ (i.e., x-pos, x-vel, y-pos, y-vel). Then, $F$ is defined by
$$ F = \begin{bmatrix}
    1 & dt & 0 & 0 \\
    0 & 1 & 0 & 0 \\
    0 & 0 & 1 & dt \\
    0 & 0 & 0 & 1 \\
\end{bmatrix}.$$

In [33]:
# dimensions
dim = 2
# timestep (1s)
dt = 1.0
# state transition matrix
F = create_state_matrix(dt, dim)
F

array([[1., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.]])

Assume $w_{k}$ has 0 mean and 0.25 variance. Sampled from a multivariate Gaussian, its covariance $Q$ is defined by
$$ Q = \begin{bmatrix}
    0.25 & 0 & 0 & 0 \\
    0 & 0.25 & 0 & 0 \\
    0 & 0 & 0.25 & 0 \\
    0 & 0 & 0 & 0.25 \\
\end{bmatrix}.$$

In [35]:
# process noise covariance
Q = np.identity(F.shape[0]) * 0.25
Q

array([[0.25, 0.  , 0.  , 0.  ],
       [0.  , 0.25, 0.  , 0.  ],
       [0.  , 0.  , 0.25, 0.  ],
       [0.  , 0.  , 0.  , 0.25]])

The measurement model describes what the sensor sees. Define the measurement model as $$ z_{k} = Hx_{k} + v_{k}$$ where
\begin{gather*}
    \text{$z_{k}$: observed measurement} \\
    \text{$H$: measurement matrix} \\
    \text{$v_{k}$: measurement noise.} \\
\end{gather*}

Assume a sensor only measures position. Then, $H$ is defined by
$$ H = \begin{bmatrix}
    1 & 0 & 0 & 0 \\
    0 & 0 & 1 & 0 \\
\end{bmatrix}.$$


In [22]:
# measurement matrix
H = create_measurement_matrix(dim)
H

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.]])

Assume $v_{k}$ has 0 mean and 1 variance. Its covariance R is defined by
$$ R = \begin{bmatrix}
    1 & 0 \\
    0 & 1 \\
\end{bmatrix}.$$

In [36]:
# measurement noise covariance
R = np.identity(H.shape[0]) * 1
R

array([[1., 0.],
       [0., 1.]])

## GM-PHD Filter

The GM-PHD filter approximates the PHD function with a Gaussian mixture. Each target is treated as a Gaussian component. The PHD is then defined as $$D_{k|k}(x) = \sum_{i=1}^{J_{k|k}}{w^{i}_{k|k}\mathcal{N}(x, m^{i}_{k|k},P^{i}_{k|k})},$$ where 
\begin{gather*}
    \text{$J_{k|k}$: number of components} \\
    \text{$w^{i}_{k|k}$: weight of component $i$} \\
    \text{$m^{i}_{k|k}$: mean of component $i$} \\
    \text{$P^{i}_{k|k}$: covariance of component $i$.}\\
\end{gather*}

In [ ]:
# number of components
num_components = 3
